In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [ ]:
data=pd.read_csv('Churn_Modelling.csv')
data.head()

In [ ]:
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)


In [ ]:
data['Geography'].unique()

In [ ]:
label_encoder=LabelEncoder()
data['Gender']=label_encoder.fit_transform(data['Gender'])

In [ ]:
from sklearn.preprocessing import OneHotEncoder
onehot_encoder=OneHotEncoder()
geo_encoded=onehot_encoder.fit_transform(data[['Geography']])

In [ ]:
onehot_encoder.get_feature_names_out(['Geography'])

In [ ]:
geo_encoded_df=pd.DataFrame(geo_encoded.toarray(),columns=onehot_encoder.get_feature_names_out(['Geography']))

In [ ]:
geo_encoded_df

In [ ]:
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

In [ ]:
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder,file)


In [ ]:
X=data.drop('Exited',axis=1)
y=data['Exited']

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)


In [ ]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.fit(X_test)

In [ ]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

### ANN Implementation

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import datetime



In [ ]:
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), #First hidden layer
    Dense(32,activation='relu'), #HL2
    Dense(1,activation='sigmoid')  #Output layer
])


In [ ]:
model.summary()

In [ ]:
opt=tf.keras.optimizers.Adam(learning_rate=0.01)
loss=tf.keras.losses.BinaryCrossentropy()

In [ ]:
model.compile(optimizer=opt,loss=loss,metrics=['accuracy'])

In [ ]:
## Set up the TensorBoard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [ ]:
#Set up Early Stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [ ]:
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback],verbose=1
)
print(history)

In [ ]:
model.save('model.h5')

In [ ]:
#Load Tensorboard Extension
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/fit/20251231-143722